# Load Data

In [1]:
import pandas as pd

data = pd.read_csv('data_processed.csv', encoding="latin-1")
data.head()

,drummer,session,id,style,bpm,beat_type,time_signature,midi_filename,audio_filename,duration,...,sambareggae,sangueo,secondline,shuffle,slow,soft,songo,soul,swing,venezuelan
0,drummer1,drummer1/eval_session,drummer1/eval_session/1,funk/groove1,138,1,4-4,drummer1/eval_session/1_funk-groove1_138_beat_...,drummer1/eval_session/1_funk-groove1_138_beat_...,27.872308,...,0,0,0,0,0,0,0,0,0,0
1,drummer1,drummer1/eval_session,drummer1/eval_session/10,soul/groove10,102,1,4-4,drummer1/eval_session/10_soul-groove10_102_bea...,drummer1/eval_session/10_soul-groove10_102_bea...,37.691158,...,0,0,0,0,0,0,0,1,0,0
2,drummer1,drummer1/eval_session,drummer1/eval_session/2,funk/groove2,105,1,4-4,drummer1/eval_session/2_funk-groove2_105_beat_...,drummer1/eval_session/2_funk-groove2_105_beat_...,36.351218,...,0,0,0,0,0,0,0,0,0,0
3,drummer1,drummer1/eval_session,drummer1/eval_session/3,soul/groove3,86,1,4-4,drummer1/eval_session/3_soul-groove3_86_beat_4...,drummer1/eval_session/3_soul-groove3_86_beat_4...,44.716543,...,0,0,0,0,0,0,0,1,0,0
4,drummer1,drummer1/eval_session,drummer1/eval_session/4,soul/groove4,80,1,4-4,drummer1/eval_session/4_soul-groove4_80_beat_4...,drummer1/eval_session/4_soul-groove4_80_beat_4...,47.987500,...,0,0,0,0,0,0,0,1,0,0


In [2]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1090 entries, 0 to 1089
Data columns (total 100 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   drummer                  1090 non-null   object 
 1   session                  1090 non-null   object 
 2   id                       1090 non-null   object 
 3   style                    1090 non-null   object 
 4   bpm                      1090 non-null   int64  
 5   beat_type                1090 non-null   int64  
 6   time_signature           1090 non-null   object 
 7   midi_filename            1090 non-null   object 
 8   audio_filename           1090 non-null   object 
 9   duration                 1090 non-null   float64
 10  split                    1090 non-null   object 
 11  onset_env_mean           1090 non-null   float64
 12  onset_env_std            1090 non-null   float64
 13  mfcc_mean                1090 non-null   float64
 14  mfcc_std               

# Split labels and features

In [3]:
data.iloc[:, 31:]

,afrobeat,afrocuban,ando,baiao,bembe,blues,bomba,bossa,brazilian,breakbeat,...,sambareggae,sangueo,secondline,shuffle,slow,soft,songo,soul,swing,venezuelan
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1085,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1086,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1087,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1088,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
data.columns

Index(['drummer', 'session', 'id', 'style', 'bpm', 'beat_type',
       'time_signature', 'midi_filename', 'audio_filename', 'duration',
       'split', 'onset_env_mean', 'onset_env_std', 'mfcc_mean', 'mfcc_std',
       'spectral_flux_mean', 'spectral_flux_std', 'spectral_contrast_mean',
       'spectral_contrast_std', 'tonnetz_mean', 'tonnetz_std', 'rms_mean',
       'rms_std', 'spectral_centroid_mean', 'spectral_centroid_std',
       'spectral_bandwidth_mean', 'spectral_bandwidth_std',
       'spectral_flatness_mean', 'spectral_flatness_std', 'tempogram_mean',
       'tempogram_std', 'afrobeat', 'afrocuban', 'ando', 'baiao', 'bembe',
       'blues', 'bomba', 'bossa', 'brazilian', 'breakbeat', 'calypso',
       'chacarera', 'chacha', 'country', 'dance', 'disco', 'dominican', 'fast',
       'folk', 'frevo', 'funk', 'fusion', 'gospel', 'groove1', 'groove10',
       'groove2', 'groove3', 'groove4', 'groove5', 'groove6', 'groove7',
       'groove8', 'groove9', 'halftime', 'hiphop', 'ijexa'

In [5]:
# seperate labels
y = data.iloc[:, 31:]
# seperate data
X = data.iloc[:, :31]
# drop unnecessary data
X = X.drop(columns=["drummer", "session", "id", "midi_filename", "audio_filename", "style"])
# time signature would be a nice feature, but they are 99% in 4/4. We might as well drop this column to pevent overfitting
X = X.drop(columns=["time_signature"])

In [6]:
# only labels
y.head()

,afrobeat,afrocuban,ando,baiao,bembe,blues,bomba,bossa,brazilian,breakbeat,...,sambareggae,sangueo,secondline,shuffle,slow,soft,songo,soul,swing,venezuelan
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [7]:
# only features
X.head()

,bpm,beat_type,duration,split,onset_env_mean,onset_env_std,mfcc_mean,mfcc_std,spectral_flux_mean,spectral_flux_std,...,rms_mean,rms_std,spectral_centroid_mean,spectral_centroid_std,spectral_bandwidth_mean,spectral_bandwidth_std,spectral_flatness_mean,spectral_flatness_std,tempogram_mean,tempogram_std
0,138,1,27.872308,test,0.820465,1.471963,-14.319676,73.690796,0.370578,0.803874,...,0.034636,0.058105,5335.603726,2688.930490,4615.955290,1007.859698,0.092827,0.133516,0.239535,0.137067
1,102,1,37.691158,test,0.673516,1.996858,-18.866692,81.949180,0.318956,0.836129,...,0.030456,0.063953,6338.191486,3683.342441,4989.509117,1157.665970,0.142638,0.155706,0.139604,0.163539
2,105,1,36.351218,test,0.807532,1.771858,-15.552315,76.598114,0.430627,0.986051,...,0.037484,0.067771,6077.338238,3792.722720,4680.404598,1377.311199,0.124579,0.146646,0.179192,0.130047
3,86,1,44.716543,test,0.714612,1.739458,-18.966784,91.778990,0.261665,0.707281,...,0.022330,0.049949,5542.225095,3417.495525,4875.089779,1265.250954,0.106046,0.138443,0.208746,0.174764
4,80,1,47.987500,test,0.617521,1.897261,-21.551086,84.417250,0.230317,0.718205,...,0.021056,0.051182,7714.574395,3887.263988,5119.254721,1091.703500,0.188627,0.157474,0.129188,0.149107


In [8]:
from sklearn.model_selection import train_test_split

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convolutional Neural Network

In [9]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Normalize images
image_generator = ImageDataGenerator(
    rescale=1.0/255
)

# Pick your label column(s)
label_column = ['N', 'D', 'G', 'C', 'A', 'H', 'M',] # add the label-columns we want to predict

# Define the data generators
train_generator = image_generator.flow_from_dataframe(
    dataframe=train_df,
    directory=path_images,
    x_col="filename",
    y_col=label_column,
    target_size=(320, 320),
    batch_size=32,
    class_mode="raw",
    color_mode="rgb" # add color mode
)

val_generator = image_generator.flow_from_dataframe(
    dataframe=val_df,
    directory=path_images,
    x_col="filename",
    y_col=label_column,
    target_size=(320, 320),
    batch_size=32,
    class_mode="raw",
    color_mode="rgb", #add color mode,
    shuffle=False,  # this is crucial for later evaluation!
)

ModuleNotFoundError: No module named 'requests'